<a href="https://colab.research.google.com/github/GiovanniPantoja12/Proyecto-final-Automatizaci-n-y-control/blob/main/Estimador_Diabetes_st.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Se instala la libreria streamlit
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.8 MB/s eta 0:00:00


In [ ]:
# Instalación de la herramienta localtunnel para exponer localmente el servidor web a través de Internet.
# Permite compartir el desarrollo web local con otras personas para pruebas o demostraciones.
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.703s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
#Aplicación del modelo y posterior creación de interfaz con streamlit
%%writefile app.py

import streamlit as st
from os import read
import sqlite3
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix;
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Conexion
conexion = sqlite3.connect('datos.db')

# Codigo sql para hacer consulta

query = """
SELECT

edad,
hipertension,
enfermedad_corazon,
bmi,
HbA1c_nivel,
nivel_glucosa_sangre,
diabetes
FROM NUEVA_TABLA

"""

dataset = pd.read_sql_query(query,conexion)

X = dataset.drop(['diabetes'], axis=1)
Y = dataset['diabetes']

X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size=0.2, random_state=10)

# Modelo RandomForest
modelo_rf = RandomForestClassifier( n_estimators=120,  # Puedes ajustar este valor
    max_depth=50,    # Puedes ajustar este valor
    min_samples_split=2,  # Puedes ajustar este valor
    min_samples_leaf=1,   # Puedes ajustar este valor
    random_state=1)

modelo_rf.fit(X_Train, Y_Train)

# Predicciones y evaluación del modelo
predicciones_rf = modelo_rf.predict(X_Test)

# accuracy
accuracy= accuracy_score(Y_Test,predicciones_rf)

# Desarrollo del codigo para streamlit

st.title("Predicción de Diabetes")
st.write('Formulario para conocer si eres propenso a diabetes')

edad = st.slider("Edad", 0, 100, 30)
hipertension = st.checkbox("Hipertensión")
enfermedad_corazon = st.checkbox("Enfermedad del corazón")

st.write('Calcula tu indice BMI')
def calcular_imc(peso, altura):
    """
    Calcula el índice de masa corporal (BMI).
    """
    if peso <= 0 or altura <= 0:
        return None
    else:
        imc = peso / (altura / 100) ** 2
        return round(imc, 2)

# Interfaz de Streamlit
st.subheader("Calculadora de BMI")

# Entradas de usuario
peso = st.number_input("Ingrese su peso (kg)", min_value=1, step=1)
altura = st.number_input("Ingrese su altura (cm)", min_value=1, step=1)

# Calcular BMI y mostrar el resultado
if st.button("Calcular IMC"):
    imc_resultado = calcular_imc(peso, altura)
    if imc_resultado is not None:
        st.success(f"Su IMC es: {imc_resultado}")
        st.write("Interpretación del BMI:")

bmi = st.slider("Índice de masa corporal (BMI)", 10.0, 50.0, 25.0)
hba1c_nivel = st.slider("Nivel de HbA1c", 4.0, 10.0, 5.0)
nivel_glucosa_sangre = st.slider("Nivel de glucosa en sangre", 50, 300, 100)

# Predecir si tiene diabetes o no
nuevos_datos = [[edad, hipertension, enfermedad_corazon, bmi, hba1c_nivel, nivel_glucosa_sangre]]
resultado = modelo_rf.predict(nuevos_datos)

# Mostrar el resultado

st.button("CANCELAR")
if st.button('DIAGNOSTICO', type="primary"):
    if resultado[0] == 0:
        st.write("No tiene diabetes.")
    else:
        st.write("Tiene diabetes.")
else:
    st.write('Rellene los datos nuevamente')


Overwriting app.py


In [ ]:
# Se inicia la aplicación Streamlit y se utiliza LocalTunnel para exponerla a Internet. Se obtiene una URL que puede ser compartida.
!streamlit run app.py --server.address=localhost &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.125.79.184
npx: installed 22 in 2.703s
your url is: https://tiny-ducks-stop.loca.lt
